**ÉTAPE 0** : préparation des données

In [7]:
import process 
import pandas as pd
import numpy as np

# Nail path : '/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DATA_Statapp.csv'
# Jerome path : 'C:\Users\33640\OneDrive\Documents\GitHub\Portfolio_clustering_project\Data\DATA_Statapp.csv'
# Mohamed path : '/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DATA_Statapp.csv'
df = pd.read_csv('/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DATA_Statapp.csv')

# Apply conversion function to 'open' and 'close' columns
df['open'] = df['open'].apply(process.safe_literal_eval)
df['close'] = df['close'].apply(process.safe_literal_eval)

# Calculate returns for each line
df['return'] = df.apply(lambda row: [(close - open) / open for open, close in zip(row['open'], row['close'])], axis=1)

new_df = df[['ticker', 'return']] # create a new data frame with the column ticker and return 

# Créons le DataFrame à partir des listes dans 'return'
# On suppose ici que 'new_df' est déjà défini et contient la colonne 'return'

# Convertir chaque liste dans la colonne 'return' en plusieurs colonnes dans le nouveau DataFrame
returns_df = pd.DataFrame(new_df['return'].tolist())

# Ajouter la colonne 'ticker' du 'new_df' au début de 'returns_df'
returns_df.insert(0, 'ticker', new_df['ticker'])

# Renommer les colonnes pour refléter qu'elles sont des rendements
returns_df.columns = ['ticker'] + [f'return_{i}' for i in range(len(returns_df.columns) - 1)]

df_cleaned = process.remove_rows_with_nan(returns_df)
df_cleaned.reset_index(drop=True, inplace=True)

process.check_nan_inf(df_cleaned)

df_cleaned.set_index('ticker', inplace=True) ## ces deux lignes sont fondamentales, ne pas les retirer !!
df.set_index('ticker', inplace=True)


There are no NaN values in the dataframe


In [8]:
import process 
lookback_window = [5000, 5250]
corr_matrix = process.correlation_matrix(lookback_window=lookback_window, df_cleaned=df_cleaned)
cluster_composition_and_centroid = process.cluster_composition_and_centroid(df_cleaned=df_cleaned, correlation_matrix=corr_matrix, number_of_clusters=20, lookback_window=lookback_window)
constituent_weights = process.constituent_weights(
    df_cleaned=df_cleaned, 
    cluster_composition=cluster_composition_and_centroid, 
    sigma=10, 
    lookback_window=lookback_window
)
cluster_return = process.cluster_return(constituent_weights=constituent_weights, df_cleaned=df_cleaned, df=df, lookback_window=lookback_window)

/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Code/process.py:49: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  A_pos = mat.applymap(lambda x: x if x >= 0 else 0)
/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Code/process.py:50: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  A_neg = mat.applymap(lambda x: abs(x) if x < 0 else 0)
/usr/local/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [9]:
markowitz_weights = process.markowitz_weights(cluster_return_res=cluster_return, constituent_weights=constituent_weights, df_cleaned=df_cleaned, df=df, lookback_window=lookback_window, eta=0.01)
final_weights = process.final_weights(markowitz_weights=markowitz_weights, constituent_weights=constituent_weights)

In [10]:
consolidated_W = process.consolidated_W(number_of_repetitions=2, lookback_window=lookback_window, df_cleaned=df_cleaned, number_of_clusters=20, sigma=10e-2, df=df, eta=0.01)

/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Code/process.py:49: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  A_pos = mat.applymap(lambda x: x if x >= 0 else 0)
/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Code/process.py:50: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  A_neg = mat.applymap(lambda x: abs(x) if x < 0 else 0)
/usr/local/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Code/process.py:49: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  A_pos = mat.applymap(lambda x: x if x >= 0 else 0)
/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project

In [11]:
evaluation_window = 10
portfolio_return = process.portfolio_returns(evaluation_window=evaluation_window, df_cleaned=df_cleaned, lookback_window=lookback_window, consolidated_W=consolidated_W, df=df)
portfolio_return

/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Code/process.py:678: RuntimeWarning: invalid value encountered in scalar divide
  portfolio_returns.loc[stock, returns] = (cl - op)/op
/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Code/process.py:678: RuntimeWarning: invalid value encountered in scalar divide
  portfolio_returns.loc[stock, returns] = (cl - op)/op
/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Code/process.py:678: RuntimeWarning: invalid value encountered in scalar divide
  portfolio_returns.loc[stock, returns] = (cl - op)/op
/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Code/process.py:678: RuntimeWarning: invalid value encountered in scalar divide
  portfolio_returns.loc[stock, returns] = (cl - op)/op
/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Code/process.py:678: RuntimeWarning: invalid value encountered in scalar divide
  portfolio_returns.loc[stock, returns] = (cl - op)/op


,portfolio return,return_5250,return_5251,return_5252,return_5253,return_5254,return_5255,return_5256,return_5257,return_5258,return_5259
return_5250,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
return_5251,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
return_5252,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
return_5253,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
return_5254,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
XLY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
XOM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
XRX,NaN,0.007356,0.006893,0.018485,0.017279,0.004296,-0.004288,-0.047018,-0.006278,0.014324,0.003091
YUM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
## we first get the open and close values for each stock 
open = pd.DataFrame(index = df_cleaned.index, columns=df_cleaned.columns[lookback_window[1]:lookback_window[1] + evaluation_window])
close = pd.DataFrame(index = df_cleaned.index, columns=df_cleaned.columns[lookback_window[1]:lookback_window[1] + evaluation_window])

for stock in open.index:
    open.loc[stock, :] = df.loc[stock, 'open'][lookback_window[1]:lookback_window[1] + evaluation_window]
    close.loc[stock, :] = df.loc[stock, 'close'][lookback_window[1]:lookback_window[1] + evaluation_window]

portfolio_returns = pd.DataFrame(index=open.columns, columns=['portfolio return'], data=np.zeros(len(open.columns)))

open

,return_5250,return_5251,return_5252,return_5253,return_5254,return_5255,return_5256,return_5257,return_5258,return_5259
ticker,,,,,,,,,,
AA,18.3,18.89,19.3,20.24,20.68,20.6,20.32,20.48,21.58,22.74
ABM,38.18,38.08,38.73,39.76,40.81,40.18,39.63,39.1,39.14,38.53
ABT,110.0,110.74,111.43,109.31,107.05,106.67,107.77,109.24,108.31,108.15
ADI,131.66,136.62,135.28,137.69,136.61,138.12,137.64999,140.0,140.71001,140.58
ADM,49.0,48.66,48.75,49.15,50.37,50.5,50.2,50.19,49.95,49.52
...,...,...,...,...,...,...,...,...,...,...
XLY,153.71001,154.64999,154.53999,156.85001,157.57001,158.60001,157.98,158.49001,157.81,157.48
XOM,36.86,37.35,37.5,40.51,41.67,40.7,39.8,38.96,38.39,40.07
XRX,21.75,21.76,22.18,23.15,23.28,23.32,22.97,22.3,22.34,22.65


In [43]:
open['return_5250']['AA']

18.3

In [44]:
for returns in portfolio_return.index:
    op, cl = 0, 0

    for stocks in consolidated_W.index:
        op += open[returns][stocks] * consolidated_W.loc[stocks, 'weight']
        cl += close[returns][stocks] * consolidated_W.loc[stocks, 'weight']

    portfolio_returns.loc[returns, 'portfolio return'] = (cl - op) / op

KeyError: 'AA'

In [ ]:
stock = 'AA'
returns = 'return_5250'
open.loc[stock, returns]
consolidated_W.loc[stock, 'weight']

In [5]:
target_day = lookback_window[1] 
cluster_target_return = process.cluster_return(constituent_weights=constituent_weights, df_cleaned=df_cleaned, df=df, lookback_window=[target_day, target_day+1])
cluster_target_return_noised = cluster_target_return

def noisy(y):

    x = y.copy()

    noise = np.random.normal(0, 1, len(y))

    for i in range(len(y)):
        x.iloc[i, 0] = y.iloc[i, 0] + noise[i]

    return x

def noised_array(y, eta):

    '''
    ----------------------------------------------------------------
    GENERAL IDEA : given an array y and a target correlation eta, 
                   compute the array with the noise  
    ----------------------------------------------------------------

    ----------------------------------------------------------------
    PARAMS : 

    - y : numpy ndarray that we want to perturb

    - eta : target correlation that we want to create between y and 
            its perturbated version

    ----------------------------------------------------------------

    ----------------------------------------------------------------
    OUTPUT : noised version of y that satisfies the targeted level 
             of correlation
    ----------------------------------------------------------------
    '''

    # We compute with a small noise 
    epsilon_std_dev = 0.1

    # Calculer la corrélation initiale
    correlation = 0

    x = y.copy()

    z = y.to_numpy()
    z = np.array([item for sublist in z for item in sublist])

    
    # Boucle pour ajuster l'écart-type du bruit jusqu'à ce que la corrélation atteigne eta
    while correlation < eta:
        # Generate a vector of Gaussian noise
        noise = np.random.normal(0, epsilon_std_dev, len(y))

        for i in range(len(y)):
            x.iloc[i, 0] = y.iloc[i, 0] + noise[i]

        w = x.to_numpy()
        w = np.array([item for sublist in w for item in sublist])
        # Calculate the new correlation
        correlation = np.corrcoef(w, z)[0, 1]

        # Adjust the standard deviation of the noise
        epsilon_std_dev += 0.01  

    return x


In [10]:
x = noised_array(y=cluster_target_return, eta=0.01).iloc[:, 0].values
x

array([-0.08953295407306291, 0.014161012997184013, 0.0505755780228499,
       -0.12672986341034503, -0.061263847822916766, -0.1415154927540358,
       0.0034097653190171095, -0.060570312485364736, 0.09970722864783374,
       0.15708617228963706, 0.04981438045306754, -0.21257699338229225,
       -0.030810652909558107, -0.006230094221206882, 0.047434122269770965,
       -0.17107143087501828, -0.06456472581336359, -0.15647180510013664,
       -0.006940247623107859], dtype=object)

In [29]:
portfolio_return = process.portfolio_returns(evaluation_window=10, df_cleaned=df_cleaned, lookback_window=lookback_window, consolidated_W=consolidated_weights)
portfolio_return

,portfolio return
return_5250,0.017398
return_5251,-0.012297
return_5252,0.020837
return_5253,0.022024
return_5254,-0.003849
return_5255,0.003957
return_5256,-0.022711
return_5257,-0.005000
return_5258,-0.006371
return_5259,-0.006529
